# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('./customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [123]:
df=data.groupby(['CustomerID','ProductName'], as_index=False)[['Quantity']].sum()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [142]:
df2=pd.pivot_table(df,values='Quantity',index=['ProductName'],columns=['CustomerID'])
df2.fillna(0, inplace=True)
df2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [143]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), index=df2.columns,columns=df2.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [144]:
top5 = distances[33].sort_values(ascending=False)[1:6]
top5.index

Int64Index([264, 3535, 3317, 2503, 3305], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [172]:
df3=df.loc[df['CustomerID'].isin(top5.index)]
df3.head() 

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [181]:
df4=df3.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
df4.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [182]:
client33=df2[[33]]
df4=client33.merge(df4, on='ProductName')
df4.head()



,33,Quantity
ProductName,,
Anchovy Paste - 56 G Tube,0.0,1
Appetizer - Mushroom Tart,0.0,1
Appetizer - Sausage Rolls,0.0,1
Apricots - Dried,1.0,1
Apricots - Halves,0.0,1


In [210]:
df5=df4[df4[33]==0].sort_values(by='Quantity', ascending=False).head()
df5

,33,Quantity
ProductName,,
Wine - Blue Nun Qualitatswein,0.0,3
Soup - Campbells Bean Medley,0.0,3
Wine - Ej Gallo Sierra Valley,0.0,3
Butter - Unsalted,0.0,3
Yoghurt Tubes,0.0,2


In [214]:
df5.to_dict()

{83973: {'Cocoa Butter': 0.0,
  'Veal - Slab Bacon': 0.0,
  'Bandage - Fexible 1x3': 0.0,
  'Ezy Change Mophandle': 0.0,
  'Pop Shoppe Cream Soda': 0.0},
 'Quantity': {'Cocoa Butter': 28,
  'Veal - Slab Bacon': 24,
  'Bandage - Fexible 1x3': 22,
  'Ezy Change Mophandle': 22,
  'Pop Shoppe Cream Soda': 20}}

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [227]:
dict={}
customers=set(df['CustomerID'].tolist())
for i in customers:
    top5 = distances[i].sort_values(ascending=False)[1:6]
    df3=df.loc[df['CustomerID'].isin(top5.index)]
    df4=df3.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
    client33=df2[[i]]
    df4=client33.merge(df4, on='ProductName')
    df5=df4[df4[i]==0].sort_values(by='Quantity', ascending=False).head()
    dict5=df5.to_dict()
    dict.update(dict5)
dict

{83973: {'Cocoa Butter': 0.0,
  'Veal - Slab Bacon': 0.0,
  'Bandage - Fexible 1x3': 0.0,
  'Ezy Change Mophandle': 0.0,
  'Pop Shoppe Cream Soda': 0.0},
 'Quantity': {'Sauce - Rosee': 15,
  'Scallop - St. Jaques': 14,
  'Chestnuts - Whole,canned': 14,
  'Bread - Roll, Soft White Round': 13,
  'Chicken - Wieners': 12},
 49159: {'Pasta - Penne, Rigate, Dry': 0.0,
  'Cookies - Assorted': 0.0,
  'Cheese - Parmesan Grated': 0.0,
  'Puree - Mocha': 0.0,
  'Rice - Jasmine Sented': 0.0},
 22536: {'Cheese - Mozzarella': 0.0,
  'Bacardi Breezer - Tropical': 0.0,
  'Chocolate - Dark': 0.0,
  'Table Cloth - 53x69 Colour': 0.0,
  'Banana - Leaves': 0.0},
 18441: {'Soupfoamcont12oz 112con': 0.0,
  'Garlic - Primerba, Paste': 0.0,
  'Lemonade - Natural, 591 Ml': 0.0,
  'Sausage - Breakfast': 0.0,
  'Beef - Texas Style Burger': 0.0},
 59399: {'Shrimp - Baby, Warm Water': 0.0,
  'Knife Plastic - White': 0.0,
  'Towels - Paper / Kraft': 0.0,
  'Nantucket - Pomegranate Pear': 0.0,
  'Cookie Chocolate Ch

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [221]:
result=pd.DataFrame.from_dict(dict)
result

,83973,Quantity,49159,22536,18441,59399,92168,86028,75791,96272,...,45042,55281,59379,75762,77810,16375,77814,20476,67582,65535
Anchovy Paste - 56 G Tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
"Appetizer - Mini Egg Roll, Shrimp",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Appetizer - Mushroom Tart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Appetizer - Sausage Rolls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricots - Dried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricots - Halves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricots Fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona - Green Tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
Artichokes - Jerusalem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Assorted Desserts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [231]:
dict={}
customers=set(df['CustomerID'].tolist())
distances = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'cosine'))), index=df2.columns,columns=df2.columns)
for i in customers:
    top5 = distances[i].sort_values(ascending=False)[1:6]
    df3=df.loc[df['CustomerID'].isin(top5.index)]
    df4=df3.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
    client33=df2[[i]]
    df4=client33.merge(df4, on='ProductName')
    df5=df4[df4[i]==0].sort_values(by='Quantity', ascending=False).head()
    dict5=df5.to_dict()
    dict.update(dict5)
dict

{83973: {'Pastry - Choclate Baked': 0.0,
  'Flour - Pastry': 0.0,
  'Lettuce - Spring Mix': 0.0,
  'Puree - Passion Fruit': 0.0,
  'Broom - Corn': 0.0},
 'Quantity': {'Grapes - Red': 83,
  'Coffee - Irish Cream': 75,
  'Muffin Chocolate Individual Wrap': 74,
  'Coconut - Shredded, Sweet': 54,
  'Blackberries': 50},
 49159: {'Turnip - White, Organic': 0.0,
  'Peas - Frozen': 0.0,
  'Beer - Blue': 0.0,
  'Lamb - Ground': 0.0,
  'Beef - Ground Medium': 0.0},
 22536: {'Hersey Shakes': 0.0,
  'Muffin Batt - Choc Chk': 0.0,
  'Bread Crumbs - Panko': 0.0,
  'Spinach - Baby': 0.0,
  'Longos - Grilled Chicken With': 0.0},
 18441: {'V8 - Berry Blend': 0.0,
  'Whmis - Spray Bottle Trigger': 0.0,
  'Mussels - Cultivated': 0.0,
  'Ecolab - Lime - A - Way 4/4 L': 0.0,
  'Milk - 2%': 0.0},
 59399: {'Ezy Change Mophandle': 0.0,
  'Anchovy Paste - 56 G Tube': 0.0,
  'Garlic - Peeled': 0.0,
  'Broom - Corn': 0.0,
  'Water - Aquafina Vitamin': 0.0},
 92168: {'Tuna - Salad Premix': 0.0,
  'Wiberg Super Cu